# Got Scotch API?

This notebook is meant to demonstrate the transformation of an annotated notebook into a HTTP API using the Jupyter kernel gateway. The result is a simple scotch recommendation engine.

The original scotch data is from [https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html](https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html).

In [3]:
import pandas as pd
import pickle
import requests
import json

## Data

We read the scotch data from a public Box URL to make this notebook more portable. This is acceptable for small, public, demo data which is what we have here.

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/jondoering/my-data/master/kaggle-titanic-train.csv')

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## API

We need to define a global `REQUEST` JSON string that will be replaced on each invocation of the API. We only care about path parameters and query string arguments, so we default those to blank here for development.

In [6]:
REQUEST = json.dumps({
    'path' : {},
    'args' : {}
})

Provide a way to get the names of all the scotches known by the model.

In [7]:
# GET /passangers
names = df['Name'].tolist()
print(json.dumps(dict(names=names)))

{"names": ["Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley (Florence Briggs Thayer)", "Heikkinen, Miss. Laina", "Futrelle, Mrs. Jacques Heath (Lily May Peel)", "Allen, Mr. William Henry", "Moran, Mr. James", "McCarthy, Mr. Timothy J", "Palsson, Master. Gosta Leonard", "Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)", "Nasser, Mrs. Nicholas (Adele Achem)", "Sandstrom, Miss. Marguerite Rut", "Bonnell, Miss. Elizabeth", "Saundercock, Mr. William Henry", "Andersson, Mr. Anders Johan", "Vestrom, Miss. Hulda Amanda Adolfina", "Hewlett, Mrs. (Mary D Kingcome) ", "Rice, Master. Eugene", "Williams, Mr. Charles Eugene", "Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)", "Masselmani, Mrs. Fatima", "Fynney, Mr. Joseph J", "Beesley, Mr. Lawrence", "McGowan, Miss. Anna \"Annie\"", "Sloper, Mr. William Thompson", "Palsson, Miss. Torborg Danira", "Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)", "Emir, Mr. Farred Chehab", "Fortune, Mr. Charles Alexander", "O'Dwyer, Miss. E

Let clients query for features about a specific scotch given its name.

In [1]:
# GET /info/:args
request = json.loads(REQUEST)
print(type(request['path']))
print(type(request))


SyntaxError: invalid syntax (<ipython-input-1-20d44933917a>, line 4)

In [10]:
# GET /passengers/:sex
request = json.loads(REQUEST)
sex = request['path'].get('sex', 'male')
names = df[df['Sex'] == sex]['Name']
# can't use to_dict because it retains numpy types which blow up when we json.dumps
print(json.dumps(dict(names=names.tolist())))

{"names": ["Braund, Mr. Owen Harris", "Allen, Mr. William Henry", "Moran, Mr. James", "McCarthy, Mr. Timothy J", "Palsson, Master. Gosta Leonard", "Saundercock, Mr. William Henry", "Andersson, Mr. Anders Johan", "Rice, Master. Eugene", "Williams, Mr. Charles Eugene", "Fynney, Mr. Joseph J", "Beesley, Mr. Lawrence", "Sloper, Mr. William Thompson", "Emir, Mr. Farred Chehab", "Fortune, Mr. Charles Alexander", "Todoroff, Mr. Lalio", "Uruchurtu, Don. Manuel E", "Wheadon, Mr. Edward H", "Meyer, Mr. Edgar Joseph", "Holverson, Mr. Alexander Oskar", "Mamee, Mr. Hanna", "Cann, Mr. Ernest Charles", "Kraeff, Mr. Theodor", "Rogers, Mr. William John", "Lennon, Mr. Denis", "Samaan, Mr. Youssef", "Panula, Master. Juha Niilo", "Nosworthy, Mr. Richard Cater", "Ostby, Mr. Engelhart Cornelius", "Woolner, Mr. Hugh", "Novel, Mr. Mansouer", "Goodwin, Master. William Frederick", "Sirayanian, Mr. Orsen", "Harris, Mr. Henry Birkhardt", "Skoog, Master. Harald", "Stewart, Mr. Albert A", "Moubarek, Master. Gerios"

Let clients request a set of scotches similar to the one named. Let clients specify how many results they wish to receive (`count`) and if they want all of the raw feature data included in the result or not (`include_features`).

In [ ]:
# # GET /scotches/:scotch/similar
# request = json.loads(REQUEST)
# name = request['path'].get('scotch', 'Talisker')
# count = request['args'].get('count', 5)
# inc_features = request['args'].get('include_features', True)

# similar = sim_df[name].order(ascending=False)
# similar.name = 'Similarity'
# df = pd.DataFrame(similar).ix[1:count+1]

# if inc_features:
#     df = df.join(features_df)
    
# df = df.reset_index().rename(columns={'Distillery': 'Name'})
# result = {
#     'recommendations' : [row[1].to_dict() for row in df.iterrows()],
#     'for': name
# }
# print(json.dumps(result))